In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from neighborhoods import nearest_neighborhood
from datetime import date
from dateutil.relativedelta import relativedelta



In [2]:
df = pd.read_csv('eviction_notices.csv', parse_dates=['File Date'], encoding = "ISO-8859-1")

/Users/jakram/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(df)

37736

In [4]:
df.head()

,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location
0,M161957,300 Block Of Park Street,San Francisco,CA,94110,2016-06-06,False,False,False,False,...,False,False,False,False,False,False,NaN,9.0,Bernal Heights,"(37.7364374645373, -122.417989910582)"
1,M162256,200 Block Of Seneca Avenue,San Francisco,CA,94112,2016-06-29,False,False,False,False,...,False,False,False,False,False,False,8/25/2021,11.0,Outer Mission,"(37.7205429316262, -122.443264852669)"
2,M162135,200 Block Of Dolores Street,San Francisco,CA,94103,2016-06-28,False,True,False,False,...,False,False,False,False,False,False,NaN,8.0,Castro/Upper Market,"(37.7652067507312, -122.426591617441)"
3,M161901,1200 Block Of 9th Avenue,San Francisco,CA,94122,2016-06-02,False,False,False,False,...,False,False,False,False,False,False,NaN,5.0,Inner Sunset,"(37.764977785911, -122.4664456379)"
4,M162428,1400 Block Of Larkin Street,San Francisco,CA,94109,2016-02-28,False,False,False,False,...,False,False,False,False,False,False,NaN,3.0,Nob Hill,"(37.7912373166567, -122.419197310472)"


In [5]:
df = df.dropna(subset=['Location'])
len(df)

36289

In [6]:
mindate = date(2007, 9, 15) # This is the minimum date for which we have home values from Zillow.
df = df[df['File Date'] > mindate]
len(df)


15888

In [7]:
list(df.select_dtypes(include=['bool']).columns)

['Non Payment',
 'Breach',
 'Nuisance',
 'Illegal Use',
 'Failure to Sign Renewal',
 'Access Denial',
 'Unapproved Subtenant',
 'Owner Move In',
 'Demolition',
 'Capital Improvement',
 'Substantial Rehab',
 'Ellis Act WithDrawal',
 'Condo Conversion',
 'Roommate Same Unit',
 'Other Cause',
 'Late Payments',
 'Lead Remediation',
 'Development',
 'Good Samaritan Ends']

In [8]:
# Function appends data from Zillow API and data file using neighborhood()  
def addPricingData_lookback(row, lookback):
    latlng = row['Location']
    lat, lng = latlng.split(',')
    lat = float(lat.replace('(','')) # Reformat lat, lng to separate arguments.
    lng = float(lng.replace(')',''))
    evictdate = row['File Date'].date() # Convert eviction date and create date keys for eviciton date and starting valuation date. 
    startdate = evictdate + relativedelta(months=-lookback)
    evictdate_key = date.strftime(evictdate,"%b%Y")
    evictdateyear = date.strftime(evictdate,"%Y")
    startdate_key = date.strftime(startdate, "%b%Y")
    neighborhood = nearest_neighborhood(lat, lng) # Create neighborhood object for lat, lng.
    name = neighborhood.name
    meanvalue = float(neighborhood.meanvalue)
    evictdatevalue = neighborhood.prices[evictdate_key]
    s = neighborhood.prices[startdate_key]
    c = neighborhood.prices[evictdate_key]
    valueincrease = (c - s)/s
    return pd.Series({'Eviction_id':row['Eviction ID'],'Lat': lat, 'Lng': lng, 'Year': evictdateyear, 'Zillow_Neighborhood':name, 'Z_2017_Value':meanvalue, 'Z_Evict_Date_Value':evictdatevalue, 'Z_Value_Increase':valueincrease}) 
    return name

def addPricingData(row):
    latlng = row['Location']
    lat, lng = latlng.split(',')
    lat = float(lat.replace('(','')) # Reformat lat, lng to separate arguments.
    lng = float(lng.replace(')',''))
    evictdate = row['File Date'].date() # Convert eviction date and create date keys for eviciton date and starting valuation date. 
    evictdate_key = date.strftime(evictdate,"%b%Y")
    evictdateyear = date.strftime(evictdate,"%Y")
    neighborhood = nearest_neighborhood(lat, lng) # Create neighborhood object for lat, lng.
    name = neighborhood.name
    meanvalue = float(neighborhood.meanvalue)
    evictdatevalue = neighborhood.prices[evictdate_key]
    return pd.Series({'Eviction_id':row['Eviction ID'],'Lat': lat, 'Lng': lng, 'Year': evictdateyear, 'Zillow_Neighborhood':name, 'Z_2017_Value':meanvalue, 'Z_Evict_Date_Value':evictdatevalue}) 
    return name



In [68]:
# aggregation_lookback = {
#     'Owner Move In' : {
#         'total_move_in' : 'sum'
#     },
#     'Condo Conversion' : {
#          'total_condo' : 'sum'
#     },
#     'Ellis Act WithDrawal' : {
#          'total_ellis' : 'sum'
#     },
#     'Non Payment' : {
#          'total_non_payment' : 'sum'
#     },
#     'Breach' : {
#          'total_breach' : 'sum'
#     },
#     'Nuisance' : {
#          'total_nuisance' : 'sum'
#     },
#     'Illegal Use' : {
#          'total_illegal_use' : 'sum'
#     },
#     'Failure to Sign Renewal' : {
#          'total_non_renewal' : 'sum'
#     },
#     'Access Denial' : {
#          'total_access_denial' : 'sum'
#     },
#     'Unapproved Subtenant' : {
#          'total_unapproved_subtenant' : 'sum'
#     },
#     'Demolition' : {
#          'total_demolition' : 'sum'
#     },
#     'Capital Improvement' : {
#          'total_capital_improvement' : 'sum'
#     },
#     'Substantial Rehab' : {
#          'total_substantial_rehab' : 'sum'
#     },
#     'Roommate Same Unit' : {
#          'total_roommate' : 'sum'
#     },
#     'Other Cause' : {
#          'total_other_cause' : 'sum'
#     },
#     'Late Payments' : {
#          'total_late_payment' : 'sum'
#     },
#     'Lead Remediation' : {
#          'total_lead' : 'sum'
#     },
#     'Development' : {
#          'total_development' : 'sum'
#     },
#     'Good Samaritan Ends' : {
#          'total_total_good_samaritan' : 'sum'
#     },
#     'Eviction ID' : {
#         'total_evictions' : 'count'
#     },
#     'Z_Value_Increase' : {
#         'average_value_increase' : 'mean'
#     },
#     'Z_2017_Value' : {
#         'current_value' : 'mean'
#     },
#     'Z_Evict_Date_Value' :{ 
#         'eviction_date_value' : 'mean'
#     }
# }

# lookbackperiod_mos = 18 # This is the number of lookback months to find change in home values prior to an eviction
# lookbackdf = df[df['File Date'] > (mindate + relativedelta(months=+lookbackperiod_mos)) ]

# # Merge the results of our calls to neighborhood() back to our dataframe.
# lookbackdf = lookbackdf.merge(lookbackdf.apply(lambda row: addPricingData_lookback(row, lookbackperiod_mos), axis = 1), left_on='Eviction ID', right_on='Eviction_id')

# # Creating a dataframe to group by neighborhood. 
# groupby_hood = lookbackdf.groupby(['Zillow_Neighborhood']).agg(aggregation_lookback)

# # Adding columns to indicate frequency of certain eviction types.
# groupby_hood['pct_move_in'] = groupby_hood[('Owner Move In','total_move_in')]/groupby_hood[('Eviction ID', 'total_evictions')]
# groupby_hood['pct_condo'] = groupby_hood[('Condo Conversion','total_condo')]/groupby_hood[('Eviction ID', 'total_evictions')]
# groupby_hood['pct_ellis'] = groupby_hood[('Ellis Act WithDrawal','total_ellis')]/groupby_hood[('Eviction ID', 'total_evictions')]
# groupby_hood['pct_suspect'] = (groupby_hood[('Owner Move In','total_move_in')] + groupby_hood[('Ellis Act WithDrawal','total_ellis')] + groupby_hood[('Condo Conversion','total_condo')] 
#                          )/groupby_hood[('Eviction ID', 'total_evictions')]



# # Drop neighborhoods that lack Zillow pricing data.
# groupby_hood = groupby_hood.dropna(subset=[('Z_Value_Increase', 'average_value_increase')]).reset_index()
# # Drop neighborhoods with small sample sizes.
# groupby_hood = groupby_hood[groupby_hood[('Eviction ID', 'total_evictions')] > 40]
# # Print data frame. 
# groupby_hood


,Zillow_Neighborhood,Owner Move In,Condo Conversion,Ellis Act WithDrawal,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,...,Development,Good Samaritan Ends,Eviction ID,Z_Value_Increase,Z_2017_Value,Z_Evict_Date_Value,pct_move_in,pct_condo,pct_ellis,pct_suspect
,,total_move_in,total_condo,total_ellis,total_non_payment,total_breach,total_nuisance,total_illegal_use,total_non_renewal,total_access_denial,...,total_development,total_total_good_samaritan,total_evictions,average_value_increase,current_value,eviction_date_value,,,,
0,Bayview,35.0,0.0,6.0,63.0,77.0,56.0,8.0,4.0,14.0,...,0.0,0.0,302,0.224686,612400.0,6.195523e+05,0.115894,0.000000,0.019868,0.135762
1,Bernal Heights,103.0,7.0,67.0,21.0,103.0,63.0,8.0,0.0,6.0,...,1.0,3.0,464,0.127983,1083700.0,9.645711e+05,0.221983,0.015086,0.144397,0.381466
2,Buena Vista Park,15.0,0.0,14.0,6.0,20.0,12.0,1.0,2.0,3.0,...,0.0,0.0,115,0.103227,1304100.0,1.138866e+06,0.130435,0.000000,0.121739,0.252174
3,Central Richmond,70.0,2.0,32.0,16.0,73.0,57.0,8.0,0.0,3.0,...,0.0,0.0,284,0.133190,1284800.0,1.158423e+06,0.246479,0.007042,0.112676,0.366197
4,Central Sunset,78.0,0.0,23.0,10.0,28.0,38.0,3.0,0.0,2.0,...,0.0,0.0,220,0.117563,994300.0,9.658918e+05,0.354545,0.000000,0.104545,0.459091
6,Corona Heights,18.0,0.0,4.0,2.0,16.0,13.0,4.0,0.0,1.0,...,0.0,0.0,70,0.112847,1325400.0,1.126299e+06,0.257143,0.000000,0.057143,0.314286
7,Cow Hollow,36.0,5.0,15.0,11.0,56.0,46.0,10.0,0.0,0.0,...,0.0,0.0,197,0.082179,1732300.0,1.514629e+06,0.182741,0.025381,0.076142,0.284264
8,Crocker Amazon,48.0,0.0,4.0,15.0,44.0,29.0,10.0,2.0,6.0,...,0.0,0.0,208,0.164779,724200.0,7.059921e+05,0.230769,0.000000,0.019231,0.250000
10,Downtown,4.0,0.0,7.0,16.0,148.0,202.0,14.0,1.0,4.0,...,0.0,0.0,403,0.110481,694600.0,6.208382e+05,0.009926,0.000000,0.017370,0.027295


In [89]:
# This dictionary can be used to count each type of eviciton during an aggregation.
baseaggregation = {
    'Owner Move In' : {
        'total_move_in' : 'sum'
    },
    'Condo Conversion' : {
         'total_condo' : 'sum'
    },
    'Ellis Act WithDrawal' : {
         'total_ellis' : 'sum'
    },
    'Non Payment' : {
         'total_non_payment' : 'sum'
    },
    'Breach' : {
         'total_breach' : 'sum'
    },
    'Nuisance' : {
         'total_nuisance' : 'sum'
    },
    'Illegal Use' : {
         'total_illegal_use' : 'sum'
    },
    'Failure to Sign Renewal' : {
         'total_non_renewal' : 'sum'
    },
    'Access Denial' : {
         'total_access_denial' : 'sum'
    },
    'Unapproved Subtenant' : {
         'total_unapproved_subtenant' : 'sum'
    },
    'Demolition' : {
         'total_demolition' : 'sum'
    },
    'Capital Improvement' : {
         'total_capital_improvement' : 'sum'
    },
    'Substantial Rehab' : {
         'total_substantial_rehab' : 'sum'
    },
    'Roommate Same Unit' : {
         'total_roommate' : 'sum'
    },
    'Other Cause' : {
         'total_other_cause' : 'sum'
    },
    'Late Payments' : {
         'total_late_payment' : 'sum'
    },
    'Lead Remediation' : {
         'total_lead' : 'sum'
    },
    'Development' : {
         'total_development' : 'sum'
    },
    'Good Samaritan Ends' : {
         'total_total_good_samaritan' : 'sum'
    },
    'Eviction ID' : {
        'total_evictions' : 'count'
    },
    'Z_2017_Value' : {
        'current_value' : 'mean'
    },
    'Z_Evict_Date_Value' :{ 
        'eviction_date_value' : 'mean'
    }
}

# Adding a key value pair for the zillow value increase for our lookback function. 
aggregation_lookback = dict(baseaggregation)
aggregation_lookback.update({ 'Z_Value_Increase' : {'average_value_increase' : 'mean'},})

lookbackperiod_mos = 18 # This is the number of lookback months to find change in home values prior to an eviction
lookbackdf = df[df['File Date'] > (mindate + relativedelta(months=+lookbackperiod_mos)) ]

# Merge the results of our calls to neighborhood() back to our dataframe.
lookbackdf = lookbackdf.merge(lookbackdf.apply(lambda row: addPricingData_lookback(row, lookbackperiod_mos), axis = 1), left_on='Eviction ID', right_on='Eviction_id')

# Creating a dataframe to group by neighborhood. 
groupby_hood = lookbackdf.groupby(['Zillow_Neighborhood'],as_index=False).agg(aggregation_lookback).reset_index()

groupby_hood[('total_non_cause','total_non_cause')] = groupby_hood[[
                                               ('Owner Move In', 'total_move_in'),
                                               ('Condo Conversion', 'total_condo'),
                                               ('Ellis Act WithDrawal', 'total_ellis'),
                                               ('Substantial Rehab', 'total_substantial_rehab'),
                                               ('Lead Remediation', 'total_lead'),
                                               ('Development', 'total_development') 
                                               ]].sum(axis=1)
groupby_hood[('non_cause_frequency','non_cause_frequency')] = (groupby_hood[('total_non_cause','total_non_cause')]/
                                                               groupby_hood[('Eviction ID','total_evictions')]
                                                              )
groupby_hood = groupby_hood.dropna(subset=[('Z_Value_Increase', 'average_value_increase')]).reset_index()
# # Drop neighborhoods with small sample sizes.
groupby_hood = groupby_hood[groupby_hood[('Eviction ID', 'total_evictions')] > 40]

groupby_hood


,level_0,index,Zillow_Neighborhood,Owner Move In,Condo Conversion,Ellis Act WithDrawal,Non Payment,Breach,Nuisance,Illegal Use,...,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Eviction ID,Z_2017_Value,Z_Evict_Date_Value,Z_Value_Increase,total_non_cause,non_cause_frequency
,,,,total_move_in,total_condo,total_ellis,total_non_payment,total_breach,total_nuisance,total_illegal_use,...,total_late_payment,total_lead,total_development,total_total_good_samaritan,total_evictions,current_value,eviction_date_value,average_value_increase,total_non_cause,non_cause_frequency
0,3,3,Bayview,35.0,0.0,6.0,63.0,77.0,56.0,8.0,...,20.0,False,0.0,0.0,302,612400.0,6.195523e+05,0.224686,41.0,0.135762
1,4,4,Bernal Heights,103.0,7.0,67.0,21.0,103.0,63.0,8.0,...,16.0,False,1.0,3.0,464,1083700.0,9.645711e+05,0.127983,184.0,0.396552
2,5,5,Buena Vista Park,15.0,0.0,14.0,6.0,20.0,12.0,1.0,...,7.0,False,0.0,0.0,115,1304100.0,1.138866e+06,0.103227,29.0,0.252174
3,6,6,Central Richmond,70.0,2.0,32.0,16.0,73.0,57.0,8.0,...,18.0,False,0.0,0.0,284,1284800.0,1.158423e+06,0.133190,104.0,0.366197
4,7,7,Central Sunset,78.0,0.0,23.0,10.0,28.0,38.0,3.0,...,10.0,False,0.0,0.0,220,994300.0,9.658918e+05,0.117563,101.0,0.459091
6,11,11,Corona Heights,18.0,0.0,4.0,2.0,16.0,13.0,4.0,...,7.0,False,0.0,0.0,70,1325400.0,1.126299e+06,0.112847,22.0,0.314286
7,12,12,Cow Hollow,36.0,5.0,15.0,11.0,56.0,46.0,10.0,...,10.0,False,0.0,0.0,197,1732300.0,1.514629e+06,0.082179,56.0,0.284264
8,13,13,Crocker Amazon,48.0,0.0,4.0,15.0,44.0,29.0,10.0,...,10.0,False,0.0,0.0,208,724200.0,7.059921e+05,0.164779,52.0,0.250000
10,15,15,Downtown,4.0,0.0,7.0,16.0,148.0,202.0,14.0,...,16.0,False,0.0,0.0,403,694600.0,6.208382e+05,0.110481,11.0,0.027295


{'Owner Move In': {'total_move_in': 'sum'}, 'Condo Conversion': {'total_condo': 'sum'}, 'Ellis Act WithDrawal': {'total_ellis': 'sum'}, 'Non Payment': {'total_non_payment': 'sum'}, 'Breach': {'total_breach': 'sum'}, 'Nuisance': {'total_nuisance': 'sum'}, 'Illegal Use': {'total_illegal_use': 'sum'}, 'Failure to Sign Renewal': {'total_non_renewal': 'sum'}, 'Access Denial': {'total_access_denial': 'sum'}, 'Unapproved Subtenant': {'total_unapproved_subtenant': 'sum'}, 'Demolition': {'total_demolition': 'sum'}, 'Capital Improvement': {'total_capital_improvement': 'sum'}, 'Substantial Rehab': {'total_substantial_rehab': 'sum'}, 'Roommate Same Unit': {'total_roommate': 'sum'}, 'Other Cause': {'total_other_cause': 'sum'}, 'Late Payments': {'total_late_payment': 'sum'}, 'Lead Remediation': {'total_lead': 'sum'}, 'Development': {'total_development': 'sum'}, 'Good Samaritan Ends': {'total_total_good_samaritan': 'sum'}, 'Eviction ID': {'total_evictions': 'count'}, 'Z_2017_Value': {'current_value':

In [ ]:
pricingdf = df.merge(df.apply(addPricingData, axis=1), left_on='Eviction ID', right_on='Eviction_id')
pricingdf

In [ ]:
aggregation_pricing = {
    'Owner Move In' : {
        'total_move_in' : 'sum'
    },
    'Condo Conversion' : {
         'total_condo' : 'sum'
    },
    'Ellis Act WithDrawal' : {
         'total_ellis' : 'sum'
    },
    'Eviction ID' : {
        'total_evictions' : 'count'
    },
    'Z_2017_Value' : {
        'current_value' : 'mean'
    },
    'Z_Evict_Date_Value' :{ 
        'eviction_date_value' : 'mean'
    }
}

# Creating a dataframe to group by year, then neighborhood. 
groupby_year_hood = pricingdf.groupby(['Zillow_Neighborhood', 'Year']).agg(aggregation_pricing).reset_index()
# Drop neighborhoods that lack Zillow pricing data.
groupby_year_hood = groupby_year_hood.dropna(subset=[('Z_2017_Value', 'current_value')]).reset_index()
groupby_year_hood

In [ ]:
aggregation_pricing = {
    'Owner Move In' : {
        'total_move_in' : 'sum'
    },
    'Condo Conversion' : {
         'total_condo' : 'sum'
    },
    'Ellis Act WithDrawal' : {
         'total_ellis' : 'sum'
    },
    'Eviction ID' : {
        'total_evictions' : 'count'
    },
    'Z_2017_Value' : {
        'current_value' : 'mean'
    },
    'Z_Evict_Date_Value' :{ 
        'eviction_date_value' : 'mean'
    }
}

# Creating a dataframe to group by year, then neighborhood. 
groupby_year = pricingdf.groupby(['Year']).agg(aggregation_pricing).reset_index()
# Drop neighborhoods that lack Zillow pricing data.
groupby_year = groupby_year.dropna(subset=[('Z_2017_Value', 'current_value')]).reset_index()
groupby_year

In [ ]:
ax = sns.regplot(y=groupby_hood[('Eviction ID', 'total_evictions')], x=groupby_hood[('Z_Value_Increase', 'average_value_increase')], data=groupby_hood)
plt.show()

In [ ]:
ax = sns.regplot(y=groupby_hood[('pct_suspect', '')], x=groupby_hood[('Z_Value_Increase', 'average_value_increase')], data=groupby_hood)
plt.show()

In [ ]:
ax = sns.lmplot(x='Lng', y='Lat', hue='Zillow_Neighborhood', size=10, data=pricingdf, fit_reg=False, legend=False)
ax.despine(left=True)

# Set title, x-axis label, y-axis label.
plt.title('Evictions in San Francisco Since 2007 with Zillow Neighborhood Property Values')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
groupby_hood_lmplot = groupby_hood.sort_values(by=('Z_Value_Increase', 'average_value_increase'))
groupby_hood_lmplot

# Set style of scatterplot
sns.set(style="darkgrid")

# Set markers
# markers = ['o','_','x']*18

# Create scatterplot of non-cause eviciton frequency with property value increase.
sns.lmplot('Z_Value_Increase', # Horizontal axis
          'pct_suspect', # Vertical axis
          data = groupby_hood_lmplot, # Data source
          fit_reg = False, # Don't fix a regression line
          hue = 'Zillow_Neighborhood', # Set color
#           markers = markers
          )

# Set title, x-axis label, y-axis label.
plt.title('Non-Cause Eviction Frequency v. Property Value Change')
plt.xlabel('Property Value Change (%)')
plt.ylabel('Frequency of Non-Cause Evicitons')

plt.show()

In [ ]:
ax = sns.barplot(x='Zillow_Neighborhood', y='pct_suspect', data=groupby_hood)
ax.set_xticklabels(labels=groupby_hood['Zillow_Neighborhood'], rotation=90)
plt.show()